# PureHDF

https://github.com/Apollo3zehn/PureHDF

## Install the .NET SDK 8.0 on Debian 11

```bash
wget https://packages.microsoft.com/config/debian/11/packages-microsoft-prod.deb -O packages-microsoft-prod.deb
sudo dpkg -i packages-microsoft-prod.deb
rm packages-microsoft-prod.deb
sudo apt-get update && sudo apt-get install -y dotnet-sdk-8.0
```

## Install the VS Code extension

[Polyglot Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode), aka `ms-dotnettools.dotnet-interactive-vscode`

## Install the PureHDF NuGet package

In [ ]:
#r "nuget: PureHDF, 1.0.0-beta.3"

## Let's go!

In [ ]:
using PureHDF;

In [ ]:
ulong path_count = 100, step_count = 1000;
double dt = 0.01, theta = 1.0, mu = 0.0, sigma = 0.1;

// How do C# programmers get through life without the normal distribution?

var normal = (Random random, double mean, double stddev) =>
{
    double x1 = 1 - random.NextDouble();
    double x2 = 1 - random.NextDouble();
    double y1 = Math.Sqrt(-2.0 * Math.Log(x1)) * Math.Cos(2.0 * Math.PI * x2);
    return y1 * stddev + mean;
};

var random = new Random();

var data = new double[path_count, step_count];

for (ulong i = 0; i < path_count; i++)
{
    data[i, 0] = 0.0d;
    for (ulong j = 1; j < step_count; j++)
    {
        double dW = normal(random, 0.0, Math.Sqrt(dt));
        data[i, j] = data[i, j - 1] + theta * (mu - data[i, j - 1]) * dt + sigma * dW;
    }
}

var dataset = new H5Dataset(data, fileDims: new ulong[] { path_count, step_count });

dataset.Attributes = new()
{
    ["comment"] = new string("This dataset contains sample paths of an Ornstein-Uhlenbeck process."),
    ["Wikipedia"] = new string("https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process"),
    ["rows"] = new string("path"),
    ["columns"] = new string("time"),
    ["dt"] = dt,
    ["θ"] = theta,
    ["μ"] = mu,
    ["σ"] = sigma
};

var file = new H5File()
{
    Attributes = new()
    {
        ["source"] = new string("https://github.com/HDFGroup/hdf5-tutorial")
    },

    ["dataset"] = dataset
};

file.Write("ou_purehdf.h5");